In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import os
import argparse
from glob import glob
from pprint import pprint

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

import sys
import urllib.request
import tarfile
import re

In [3]:
tf.test.gpu_device_name() 

'/device:GPU:0'

In [0]:
os.chdir("/content/drive/My Drive/eva/session14/")

In [0]:
FLAGS=tf.app.flags.FLAGS

In [0]:
# Basic model parameters.
tf.app.flags.DEFINE_integer('batch_size', 128,
                            """Number of images to process in a batch.""")


In [0]:
TOWER_NAME = 'tower'

### Input

In [0]:


DATA_URL = 'https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'


def maybe_download_and_extract(data_dir='data'):
    """Download and extract the tarball from Alex's website."""
    dest_directory = data_dir
    os.makedirs(dest_directory, exist_ok=True)
    filename = DATA_URL.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)

    if not os.path.exists(filepath):
        def _progress(count, block_size, total_size):
            sys.stdout.write('\r>> Downloading {} {:.2%}%'.format(
                filename, float(count * block_size) / float(total_size) * 100.0))
            sys.stdout.flush()

        filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
        statinfo = os.stat(filepath)
        print('\nSuccessfully downloaded', filename, statinfo.st_size, 'bytes.')

    extracted_dir_path = os.path.join(dest_directory, 'cifar-10-batches-bin')
    if not os.path.exists(extracted_dir_path):
        tarfile.open(filepath, 'r:gz').extractall(dest_directory)

In [0]:
batch_size =128
data_dir='data/cifar/'
model_dir="models/cifar"
NUM_CLASSES=10

mode = tf.estimator.ModeKeys.TRAIN

In [0]:

def _variable_on_cpu(name, shape, initializer):
  """Helper to create a Variable stored on CPU memory.
  Args:
    name: name of the variable
    shape: list of ints
    initializer: initializer for Variable
  Returns:
    Variable Tensor
  """
  with tf.device('/gpu:0'):
    dtype = tf.float32
    var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
  return var

In [0]:

def _variable_with_weight_decay(name, shape, stddev, wd):
  """Helper to create an initialized Variable with weight decay.
  Note that the Variable is initialized with a truncated normal distribution.
  A weight decay is added only if one is specified.
  Args:
    name: name of the variable
    shape: list of ints
    stddev: standard deviation of a truncated Gaussian
    wd: add L2Loss weight decay multiplied by this float. If None, weight
        decay is not added for this Variable.
  Returns:
    Variable Tensor
  """
  dtype = tf.float32
  var = _variable_on_cpu(
      name,
      shape,
      tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
  if wd is not None:
    weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
    tf.add_to_collection('losses', weight_decay)
  return var


In [0]:

def _activation_summary(x):
  """Helper to create summaries for activations.
  Creates a summary that provides a histogram of activations.
  Creates a summary that measures the sparsity of activations.
  Args:
    x: Tensor
  Returns:
    nothing
  """
  # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
  # session. This helps the clarity of presentation on tensorboard.
  tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
  tf.summary.histogram(tensor_name + '/activations', x)
  tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))


### Model

In [0]:

def l2_regularizer(wd):
    def _l2_regularizer(var, name=None):
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name=name)
        tf.add_to_collection('losses', weight_decay)
        return weight_decay
    return _l2_regularizer

In [0]:
def conv_block(inputs, filters, kernel_size, name):
    """Builds a ConvBlock as seen in the section 2.3 slides.

    Args:
        inputs:
        filters: {N}
        kernel_size: {KS}
        name:

    Assumes we are using the default data format: NHWC

    For the stuff below, let:
        PS = pool_size
        KS = kernel_size
        S = stride for pooling

    Output shapes can be computed with standard formulas, given stride=1:
        Output height H <= (H - KS + 1 - PS)/S + 1
        Output width: W <= (W - KS + 1 - PS)/S + 1
    """
    with tf.variable_scope(name, 'conv_block'):
        x = tf.layers.Conv2D(
            filters, kernel_size,
            padding='same',
            use_bias=False)(inputs)

            
        x = tf.layers.BatchNormalization(
            epsilon=1e-5,
            fused=True,
            name='batch_norm')(x, training=True)
        x = tf.nn.relu(x, name='relu')
        x = tf.layers.MaxPooling2D(
            pool_size=3,
            strides=2,
            padding='same',
            name='max_pool')(x)
        return x

In [0]:
#!/usr/bin/python
# -*- coding: utf-8 -*-


def inference(image_batch, batch_size=128):  # input batch of images and size
    """Build the CIFAR-10 model.
    
    Args:
      image_batch: Images returned from distorted_inputs() or inputs().
      batch_size: (int) number of examples per batch.
    
    Returns:
      Logits.
    """

    # ## Prep
    # conv1

    with tf.variable_scope('conv1') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[5, 5, 3,
                64], stddev=5e-2, wd=None)
        conv1 = tf.nn.conv2d(image_batch, kernel, [1, 1, 1, 1],
                             padding='SAME')  # conv1
        biases1 = _variable_on_cpu('biases1', [64],
                                   tf.constant_initializer(0.0))
        pre_activation1 = tf.nn.bias_add(conv1, biases1)
        conv1 = tf.nn.relu(pre_activation1, name=scope.name)  # Relu 1
        _activation_summary(conv1)

        # norm1

        norm1 = tf.nn.lrn(
            conv1,
            4,
            bias=1.0,
            alpha=0.001 / 9.0,
            beta=0.75,
            name='norm1',
            )

#####Layer 1

# conv2

    with tf.variable_scope('conv2') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[5, 5,
                64, 64], stddev=5e-2, wd=None)
        conv2 = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME'
                             )  # Conv2
        biases2 = _variable_on_cpu('biases2', [64],
                                   tf.constant_initializer(0.1))
        pre_activation2 = tf.nn.bias_add(conv2, biases2)
        conv2 = tf.nn.relu(pre_activation2, name=scope.name)  # Relu 2
        _activation_summary(conv2)

        # norm2

        norm2 = tf.nn.lrn(
            conv2,
            4,
            bias=1.0,
            alpha=0.001 / 9.0,
            beta=0.75,
            name='norm2',
            )

        # pool1

        pool1 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1,
                               2, 2, 1], padding='SAME', name='pool1')

    # # res1 Layer 1

    with tf.variable_scope('conv3') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[5, 5,
                64, 64], stddev=5e-2, wd=None)
        conv3 = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME'
                             )  # Conv 3
        biases3 = _variable_on_cpu('biases3', [64],
                                   tf.constant_initializer(0.1))
        pre_activation3 = tf.nn.bias_add(conv3, biases3)
        conv3 = tf.nn.relu(pre_activation3, name=scope.name)  # Relu 3
        _activation_summary(conv3)

        # norm3

        norm3 = tf.nn.lrn(
            conv3,
            4,
            bias=1.0,
            alpha=0.001 / 9.0,
            beta=0.75,
            name='norm3',
            )

    # ## Resnet 2 Layer 1

    with tf.variable_scope('conv4') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[5, 5,
                64, 64], stddev=5e-2, wd=None)
        conv4 = tf.nn.conv2d(norm3, kernel, [1, 1, 1, 1], padding='SAME'
                             )  # Conv2
        biases4 = _variable_on_cpu('biases4', [64],
                                   tf.constant_initializer(0.1))
        pre_activation4 = tf.nn.bias_add(conv4, biases4)
        conv4 = tf.nn.relu(pre_activation4, name=scope.name)  # Relu 2
        _activation_summary(conv4)

        # norm2

        norm4 = tf.nn.lrn(
            conv4,
            4,
            bias=1.0,
            alpha=0.001 / 9.0,
            beta=0.75,
            name='norm2',
            )

        # # Do addition

        # pool4

        #pool4 = tf.nn.max_pool(norm4, ksize=[1, 3, 3, 1], strides=[1,
         #                      2, 2, 1], padding='SAME', name='pool1')

    # #################

    with tf.variable_scope('flatten') as scope:
    # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.keras.layers.Flatten()(norm4)
        dim = reshape.get_shape()[1].value
        weights = _variable_with_weight_decay('weights', shape=[dim, 384],
                                              stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
        _activation_summary(local3)

  # local4
    with tf.variable_scope('local4') as scope:
        weights = _variable_with_weight_decay('weights', shape=[384, 192],
                                              stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
        _activation_summary(local4)

  # linear layer(WX + b),
  # We don't apply softmax here because
  # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
  # and performs the softmax internally for efficiency.
    with tf.variable_scope('softmax_linear') as scope:
        weights = _variable_with_weight_decay('weights', [192, NUM_CLASSES],
                                              stddev=1/192.0, wd=None)
        biases = _variable_on_cpu('biases', [NUM_CLASSES],
                                  tf.constant_initializer(0.0))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
        _activation_summary(softmax_linear)

        return softmax_linear


In [0]:
def l2loss(logits, labels):
    """Add L2Loss to all the trainable variables.
    Add summary for "Loss" and "Loss/avg".
    Args:
      logits: Logits from inference().
      labels: Labels from distorted_inputs or inputs(). 1-D tensor
              of shape [batch_size]
    Returns:
      Loss tensor of type float.
    """
    # Calculate the average cross entropy loss across the batch.
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels, logits=logits, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    # The total loss is defined as the cross entropy loss plus all of the weight
    # decay terms (L2 loss).
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [0]:
def _add_loss_summaries(total_loss):
  """Add summaries for losses in CIFAR-10 model.
  Generates moving average for all losses and associated summaries for
  visualizing the performance of the network.
  Args:
    total_loss: Total loss from loss().
  Returns:
    loss_averages_op: op for generating moving averages of losses.
  """
  # Compute the moving average of all individual losses and the total loss.
  loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
  losses = tf.get_collection('losses')
  loss_averages_op = loss_averages.apply(losses + [total_loss])

  # Attach a scalar summary to all individual losses and the total loss; do the
  # same for the averaged version of the losses.
  for l in losses + [total_loss]:
    # Name each loss as '(raw)' and name the moving average version of the loss
    # as the original loss name.
    tf.summary.scalar(l.op.name + ' (raw)', l)
    tf.summary.scalar(l.op.name, loss_averages.average(l))

  return loss_averages_op

In [0]:
def accuracy(logits,labels):

    acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(labels, 1), 
                                  predictions=tf.argmax(logits,1))
    
    tf.add_to_collection('accuracy', acc)
    
    return acc, acc_op
    

In [0]:
def train(total_loss, batch_size):
  """Train CIFAR-10 model.
  Create an optimizer and apply to all trainable variables. Add moving
  average for all trainable variables.
  Args:
    total_loss: Total loss from loss().
    global_step: Integer Variable counting the number of training steps
      processed.
  Returns:
    train_op: op for training.
  """
  # Variables that affect learning rate.

  global_step=tf.train.get_or_create_global_step()
  num_batches_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / FLAGS.batch_size
  decay_steps = int(num_batches_per_epoch * NUM_EPOCHS_PER_DECAY)

  # Decay the learning rate exponentially based on the number of steps.
  lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE,
                                  global_step,
                                  decay_steps,
                                  LEARNING_RATE_DECAY_FACTOR,
                                  staircase=True)
  tf.summary.scalar('learning_rate', lr)

  # Generate moving averages of all losses and associated summaries.
  loss_averages_op = _add_loss_summaries(total_loss)

  update_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)

  # Compute gradients.
  with tf.control_dependencies([loss_averages_op]):
    opt = tf.train.GradientDescentOptimizer(lr)
    
    grads = opt.compute_gradients(total_loss)

  # Apply gradients.
  apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

  # Add histograms for trainable variables.
  for var in tf.trainable_variables():
    tf.summary.histogram(var.op.name, var)

  # Add histograms for gradients.
  for grad, var in grads:
    if grad is not None:
      tf.summary.histogram(var.op.name + '/gradients', grad)

  # Track the moving averages of all trainable variables.
  variable_averages = tf.train.ExponentialMovingAverage(
      MOVING_AVERAGE_DECAY, global_step)
  with tf.control_dependencies([apply_gradient_op]):
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

  return variables_averages_op

In [0]:
def train(total_loss, batch_size):
    global_step = tf.train.get_or_create_global_step()
    num_examples_per_epoch_for_train = 50000 # Number of training samples
    num_epochs_per_decay = 350.0
    num_batches_per_epoch = num_examples_per_epoch_for_train / batch_size  # Calculate number of batches for each epoch
    decay_steps = int(num_batches_per_epoch * num_epochs_per_decay)
    lr = tf.train.exponential_decay(
        0.1, global_step, decay_steps,
        decay_rate=0.1, staircase=True)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS) # To ensure anything in below code block is run already
    with tf.control_dependencies(update_ops):
        train_op = tf.contrib.layers.optimize_loss(
            loss=total_loss,
            global_step=global_step,
            learning_rate=lr,
            optimizer='SGD')         
    return train_op

In [0]:
# CIFAR-10 consists of 50K training examples and 10K eval examples.
# Each image has size 32x32 (and depth 3 for RGB).
CIFAR_TRAIN_SIZE = 50000
CIFAR_EVAL_SIZE = 10000
CIFAR_IMAGE_SIZE = 32

In [0]:



def input_fn(data_dir,batch_size): # input parameter dir of cifar 10 data and batch size
    # The src of data will be downloaded to pwd when running first time
    filenames = glob(os.path.join(data_dir, 'cifar-10-batches-bin', 'data_batch_*.bin')) # List all filenames inside cifar-10-batches-bin directory and list all filenames matching data_batch_*.bin 
    pprint(filenames)

    depth = 3
    height = width = CIFAR_IMAGE_SIZE
    label_bytes = 1
    image_bytes = height * width * depth
    # Every record consists of a label followed by the image,
    # with a fixed number of bytes for each.
    record_bytes = label_bytes + image_bytes

    def decode_line(value): # Convert binary record into image tensor and it's corresponding label
        """Additional processing to perform on each line in dataset."""
        record_bytes = tf.decode_raw(value, tf.uint8) # 1. Convert each byte into integer representation
        # The first bytes represent the label, which we convert from uint8->int32.
        label = tf.to_int32(tf.strided_slice(record_bytes, [0], [label_bytes]))
        # The remaining bytes after the label represent the image, which we reshape
        # from [depth * height * width] to [depth, height, width].
        depth_major = tf.reshape(
            tf.strided_slice(record_bytes, [label_bytes], [label_bytes + image_bytes]),
            [depth, height, width])
        # Convert from [depth, height, width] to [height, width, depth].
        uint8image = tf.transpose(depth_major, [1, 2, 0])
        reshaped_image = tf.cast(uint8image, tf.float32)
        # Randomly flip the image horizontally.
        distorted_image = tf.image.random_flip_left_right(reshaped_image)
        # Subtract off the mean and divide by the variance of the pixels.
        float_image = tf.image.per_image_standardization(distorted_image)
        # Set the shapes of tensors.
        float_image.set_shape([height, width, 3])
        label.set_shape([1])
        return float_image, label

    # Repeat infinitely.
    dataset = tf.data.FixedLengthRecordDataset(filenames, record_bytes).repeat() # expects list of filenames and number of bytes each record takes 
    dataset = dataset.map(decode_line, num_parallel_calls=batch_size) # Converts binary record into image tensor

    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(CIFAR_TRAIN_SIZE * min_fraction_of_examples_in_queue)
    dataset = dataset.shuffle(buffer_size=min_queue_examples + 3 * batch_size) # Customise data shuffle in input pipeline

    # Batch it up.
    dataset = dataset.batch(batch_size) # Set batch size
    iterator = dataset.make_one_shot_iterator() # Iterate over batches returning image batch, label pairs
    image_batch, label_batch = iterator.get_next() # Retrieves sequence of batches
    # Ensure we don't have any shape dimensions equal to None...
    image_batch.set_shape([batch_size, height, width, 3])
    label_batch = tf.squeeze(label_batch)
    return image_batch, label_batch

In [0]:


def model_fn(features, labels, mode):
     logits = inference(
        image_batch=features,
        batch_size=batch_size)
    
     loss =l2loss(logits, labels)
     
     train_op =train(loss, batch_size=batch_size)

     if mode == tf.estimator.ModeKeys.TRAIN:
        logging_hook = tf.train.LoggingTensorHook({'loss': loss}, every_n_iter=1)
        return tf.estimator.EstimatorSpec(
            mode,
            loss=loss,
            train_op=train_op,
            
            training_hooks=[logging_hook])



In [0]:
def main():
    # Ensure data_dir has dataset and model_dir is cleared before training.
    maybe_download_and_extract(data_dir=data_dir) # Download data if not exist in data dir specified
    if tf.gfile.Exists(model_dir):
        tf.gfile.DeleteRecursively(model_dir)
    tf.gfile.MakeDirs(model_dir)  # Clear model dir before instatiating

    
    classifier = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir=model_dir)
    classifier.train(
        input_fn=lambda: input_fn(data_dir, batch_size),
        steps=20) # Train classifier calls model_fn build entire model graph  look past input function and do training updates used simple callable a funtion can be called without param by wrapping in lambda
   
    


In [25]:
if __name__ == '__main__':
  main()

I0816 12:34:36.182737 140185262561152 estimator.py:1790] Using default config.
I0816 12:34:36.185557 140185262561152 estimator.py:209] Using config: {'_model_dir': 'models/cifar', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7f224e4828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
W0816 12:34:36.1995

['data/cifar/cifar-10-batches-bin/data_batch_5.bin',
 'data/cifar/cifar-10-batches-bin/data_batch_1.bin',
 'data/cifar/cifar-10-batches-bin/data_batch_2.bin',
 'data/cifar/cifar-10-batches-bin/data_batch_4.bin',
 'data/cifar/cifar-10-batches-bin/data_batch_3.bin']


W0816 12:34:37.743663 140185262561152 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

I0816 12:34:37.918793 140185262561152 estimator.py:1147] Done calling model_fn.
I0816 12:34:37.922474 140185262561152 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
W0816 12:34:38.063102 140185262561152 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
I0816 12:34:38.159185 140185262